In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark import SparkContext, SparkConf, SparkSession

ImportError: ignored

In [3]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [5]:
rdd = spark.sparkContext.textFile("/data/user_events_part/events_user_dt_sample.txt")

In [11]:
domain_difftime_rdd = rdd.map(lambda text : text.split('\t')[2:])

# delete www. in domains and diff_time:999 -> int(999)
prepared_domain_difftime_rdd = domain_difftime_rdd.map(lambda x : (x[0].replace('://www.', '://'), int(x[1].split(':')[1]))) 
prepared_domain_difftime_rdd.take(2)

[('http://worldoftanks.ru/ru/news/pc-browser/11/autumn_armored_maneuvers/',
  9745),
 ('http://motorpage.ru/feedbacks/', 100)]

In [18]:
from operator import add

agregated_key_rdd = prepared_domain_difftime_rdd.aggregateByKey(0, add, add)
agregated_key_rdd.take(2)

[('http://championat.com/football/article-207255-futbolisty-v-instagrame-luchshee-za-nedelju.html',
  24),
 ('http://lifenews.ru/news/142341', 3)]

In [38]:
domains_rdd = agregated_key_rdd.map(lambda x : (x[0].split('://')[1].split('/')[0], x[1])).groupByKey()
quantile_rdd = domains_rdd.map(lambda x : (x[0], np.median(list(x[1])), np.quantile(list(x[1]), 0.75)))
median_sorted_rdd = quantile_rdd.map(lambda x : (x[1], x)).sortByKey(ascending=False)

In [39]:
top_10 = median_sorted_rdd.take(10)
for result in top_10:
    print(*result[1], sep=' ')

bashmedia.info 14975.0 14975.0
ridus.ru 11308.0 11308.0
evening-kazan.ru 7971.0 7971.0
furfur.me 4836.0 7160.0
tech.obozrevatel.com 4246.0 4246.0
filmz.ru 4243.0 4243.0
instaforex.com 3828.0 3828.0
readovka.ru 2661.0 9042.5
tanci.tnt-online.ru 2238.0 2238.0
life.panorama.am 2192.0 2192.0
